In [332]:
import sqlite3 as dbms

conn = dbms.connect("students.sqlite3")  # Как оригинально!
cursor = conn.cursor()

In [333]:
with open("11.drop_ddl.sql", 'r', encoding='utf-8') as f:
    drop_ddl = f.read()
with open("11.create_ddl.sql", 'r', encoding='utf-8') as f:
    create_ddl = f.read()

In [334]:
if True:
    cursor.executescript(drop_ddl)
    conn.commit()
if True:
    cursor.executescript(create_ddl)
    conn.commit()

In [335]:
import sqlalchemy
from sqlalchemy import Table, Column, Integer, String, MetaData, ForeignKey
from sqlalchemy.orm import sessionmaker, relationship
from sqlalchemy.ext.declarative import declarative_base

DeclBase = declarative_base()

engine = sqlalchemy.create_engine('sqlite:///students.sqlite3', echo=False)  # echo=True для логгинга
Session = sessionmaker(bind=engine)
session = Session()

In [336]:
class Program(DeclBase):
    __tablename__ = 'programs'
    id = Column(Integer, primary_key=True)
    name = Column(String)

    students = relationship("Student", backref="program")
    programs_courses = relationship("Program_course", backref="program")

    def __init__(self, name):
        self.name = name

class Student(DeclBase):
    __tablename__ = 'students'
    id = Column(Integer, primary_key=True)
    card = Column(String)
    surname = Column(String)
    name = Column(String)
    patronymic = Column(String)
    
    program_id = Column(Integer, ForeignKey('programs.id'))
    marks = relationship("Mark", backref="student")
    
    def __init__(self, card, surname, name, patronymic, program):
        self.card = card
        self.surname = surname
        self.name = name
        self.patronymic = patronymic
        self.program = program
        
class Course(DeclBase):
    __tablename__ = 'courses'
    id = Column(Integer, primary_key=True)
    name = Column(String)
    programs_courses = relationship("Program_course", backref="course")
    marks = relationship("Mark", backref="course")
    teacher = "Горячко Евгений Евгеньевич" #по дефолту за всё отвечает Горячко
    def __init__(self, name, teacher):
        self.teacher = teacher #каждую дисциплину ведёт ровно один преподаватель
        self.name = name
        
class Program_course(DeclBase):
    __tablename__ = 'programs_courses'
    semester_number = Column(Integer, primary_key=True)
    course_id = Column(Integer, ForeignKey('courses.id'), primary_key=True)
    program_id = Column(Integer, ForeignKey('programs.id'), primary_key=True)
    def __init__(self, semester_number, course, program):
        self.semester_number = semester_number
        self.course = course
        self.program = program
        
class Mark(DeclBase):
    __tablename__ = 'marks'
    mark = Column(Integer)
    course_id = Column(Integer, ForeignKey('courses.id'), primary_key=True)
    student_id = Column(Integer, ForeignKey('students.id'), primary_key=True)
    def __init__(self, mark, student, course):
        self.mark = mark
        self.student = student
        self.course = course

In [337]:
se = Program("Теоретическая Физика")

st1 = Student("102001", "Зибинский", "Алексей", "Леонидович", se)
st2 = Student("102002", "Гусев", "Влад", "Игоревич", se)
c1 = Course("Алгебра", "Горячко Евгений Евгеньевич")
c2 = Course("Математический Анализ", "Жуков Игорь Борисович")
pc1 = Program_course("1", c1, se)
pc2 = Program_course("1", c2, se)
m1 = Mark("5", st1, c1)
m2 = Mark("5", st1, c2)
m3 = Mark("4", st2, c1)
m4 = Mark("4", st2, c2)


session.add_all([st1, st2, c1, c2, pc1, pc2, m1, m2, m3, m4]) 
session.commit()

In [338]:
se = Program("Программная инженерия")

st1 = Student("002004", "Рабош", "Артём", "Леонидович", se)
st2 = Student("002003", "Табуев", "Анри", "Сосланович", se)
c1 = Course("Дискретная математика для программистов", "Новиков Фёдор Александрович")
pc1 = Program_course("2", c1, se)
m1 = Mark("2", st1, c1)
m2 = Mark("2", st2, c1)


session.add_all([st1, st2, c1, pc1, m1, m2]) 
session.commit()

In [339]:
import sys

print("Программы и студенты")
for p in session.query(Program):
    print("")
    print("Программа и изучаемые дисциплины: ", p.name)
    print("Студенты:")
    for s in p.students:
        print("- ", s.surname,s.name, s.patronymic)
    print("Дисциплины:")
    for s in p.programs_courses:
        print("- ",s.course.name,"в семестре номер", s.semester_number, "с преподавателем", s.course.teacher)
print("")
print("Оценки:")
for p in session.query(Student):
    print("")
    print("Студент", p.surname, p.name, "и его оценки:")
    for s in p.marks:
        print("- ", s.mark, "по дисциплине", s.course.name)

Программы и студенты

Программа и изучаемые дисциплины:  Теоретическая Физика
Студенты:
-  Зибинский Алексей Леонидович
-  Гусев Влад Игоревич
Дисциплины:
-  Алгебра в семестре номер 1 с преподавателем Горячко Евгений Евгеньевич
-  Математический Анализ в семестре номер 1 с преподавателем Жуков Игорь Борисович

Программа и изучаемые дисциплины:  Программная инженерия
Студенты:
-  Рабош Артём Леонидович
-  Табуев Анри Сосланович
Дисциплины:
-  Дискретная математика для программистов в семестре номер 2 с преподавателем Новиков Фёдор Александрович

Оценки:

Студент Зибинский Алексей и его оценки:
-  5 по дисциплине Алгебра
-  5 по дисциплине Математический Анализ

Студент Гусев Влад и его оценки:
-  4 по дисциплине Алгебра
-  4 по дисциплине Математический Анализ

Студент Рабош Артём и его оценки:
-  2 по дисциплине Дискретная математика для программистов

Студент Табуев Анри и его оценки:
-  2 по дисциплине Дискретная математика для программистов
